In [1]:
from ecies.utils import generate_key
from ecies import encrypt, decrypt
import hashlib
# Step 1: Generate private and public keys
private_key = generate_key()
private_key_hex = private_key.to_hex()
public_key_hex = private_key.public_key.format(compressed=False).hex()  # Public key in hex
print("Step 1: Key Generation")
print("Private Key (hex):", private_key_hex)
print("Public Key (hex):", public_key_hex)

Step 1: Key Generation
Private Key (hex): 1c70ad9c314880c182ed9d757ee6e44bbeb7538fdd21f16358c2842219e8f148
Public Key (hex): 04e866ebb60b0d98970ee9368fb61ada619019fc6ead91df5b8025e70932f815057e3be24b6965c3d39533b03285d77af35e825523a54ef89742c0767de3c70622


In [2]:
# Step 2: User input for message to encrypt
message = input("\nStep 2: Enter the message to encrypt: ").encode('utf-8')
print("\nOriginal Message:", message.decode())


Step 2: Enter the message to encrypt:  check 1 check 2



Original Message: check 1 check 2


In [3]:
# Step 3: Encrypt the message using the encrypt() function
print("\nStep 3: Encrypting the message (ECIES)")
encrypted_message = encrypt(public_key_hex, message)
# ECIES encryption process - Detailed
print("\nDetailed Encryption Process:")
# 3.1: The sender uses the receiver's public key and their private key to derive the shared secret (ECDH)
# Correctly passing the public key in bytes format
receiver_public_key = bytes.fromhex(public_key_hex)  # Receiver's public key in bytes
sender_shared_secret = private_key.ecdh(receiver_public_key)
print("Shared Secret derived (ECDH):", sender_shared_secret.hex())
# 3.2: KDF to derive the encryption key and MAC key from the shared secret
derived_key = hashlib.sha256(sender_shared_secret).digest()  # Use SHA-256 as KDF (for simplicity)
encryption_key = derived_key[:16]  # AES encryption key (first 16 bytes)
mac_key = derived_key[16:]  # MAC key (next 16 bytes)
print("Derived Key (SHA-256):", derived_key.hex())
print("Encryption Key (AES):", encryption_key.hex())
print("MAC Key:", mac_key.hex())
# 3.3: AES encryption is done using the encryption key (AES encryption) and the message is encrypted
print("\nEncrypted Message (hex):", encrypted_message.hex())


Step 3: Encrypting the message (ECIES)

Detailed Encryption Process:
Shared Secret derived (ECDH): 54eb412ee1dd664e04f3d0dc23a71cad58c162e164f50c308eb783802c9b84ee
Derived Key (SHA-256): 5730c6201a30f5048eb48c3171912c7c45039215f732662c96f233f2efd34422
Encryption Key (AES): 5730c6201a30f5048eb48c3171912c7c
MAC Key: 45039215f732662c96f233f2efd34422

Encrypted Message (hex): 042d0ed8bd58d4f9e6e4cd9a83f8ec3665ca897516c5196d9a477fcfe238d12899977af8a196f6cb0d0a01f137df3a93cb4b671a4dd979b4501051814be47de6733c967620cbed18b0af654b2ac4a8de804b6c244a6b8d6755f9837410a6c711f06b57a4ff458cd4b03adbaedffb0b16


In [4]:
# Step 4: Decrypt the message using the decrypt() function
print("\nStep 4: Decrypting the message (ECIES)")
# ECIES decryption process - Detailed
decrypted_message = decrypt(private_key_hex, encrypted_message)
print("\nDetailed Decryption Process:")
# 4.1: The receiver uses their private key and the sender's public key to derive the shared secret (ECDH)
receiver_shared_secret = private_key.ecdh(receiver_public_key)
print("Shared Secret derived (ECDH - receiver):", receiver_shared_secret.hex())
# 4.2: KDF to derive the decryption key and MAC key from the shared secret
derived_key_receiver = hashlib.sha256(receiver_shared_secret).digest()  # Use SHA-256 as KDF
encryption_key_receiver = derived_key_receiver[:16]  # AES encryption key (first 16 bytes)
mac_key_receiver = derived_key_receiver[16:]  # MAC key (next 16 bytes)
print("Derived Key (SHA-256 - receiver):", derived_key_receiver.hex())
print("Encryption Key (AES - receiver):", encryption_key_receiver.hex())
print("MAC Key (receiver):", mac_key_receiver.hex())
# 4.3: The receiver checks the MAC to verify integrity and authenticity (this is a simplified model, actual ECIES checks it)
# 4.4: AES decryption is done using the derived encryption key
print("\nDecrypted Message:", decrypted_message.decode())


Step 4: Decrypting the message (ECIES)

Detailed Decryption Process:
Shared Secret derived (ECDH - receiver): 54eb412ee1dd664e04f3d0dc23a71cad58c162e164f50c308eb783802c9b84ee
Derived Key (SHA-256 - receiver): 5730c6201a30f5048eb48c3171912c7c45039215f732662c96f233f2efd34422
Encryption Key (AES - receiver): 5730c6201a30f5048eb48c3171912c7c
MAC Key (receiver): 45039215f732662c96f233f2efd34422

Decrypted Message: check 1 check 2
